In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten

inp = Input(shape=(240, 320, 1))

n_filters = 512
max_p = inp

for _ in range(3):
    conv = Conv2D(filters=n_filters, kernel_size=5, activation='relu')(max_p)
    max_p = MaxPooling2D(pool_size=(2, 2))(conv)
    
    n_filters //= 2

conv = Conv2D(filters=n_filters, kernel_size=3, activation='relu')(max_p)
max_p = MaxPooling2D(pool_size=(2, 2))(conv)

flatten = Flatten()(max_p)
dense = Dense(128, activation='relu')(flatten)
# out = Dense(14, activation='relu')(dense)

head_out = Dense(1, activation='relu', name='head_out')(dense)
neck_out = Dense(1, activation='relu', name='neck_out')(dense)
sts_out = Dense(1, activation='relu', name='sts_out')(dense)
arm_out = Dense(1, activation='relu', name='arm_out')(dense)
stw_out = Dense(1, activation='relu', name='shoulder_to_wrist_out')(dense)
torso_out = Dense(1, activation='relu', name='torso_out')(dense)
bicep_out = Dense(1, activation='relu', name='bicep_out')(dense)
wrist_out = Dense(1, activation='relu', name='wrist_out')(dense)
chest_out = Dense(1, activation='relu', name='chest_out')(dense)
waist_out = Dense(1, activation='relu', name='waist_out')(dense)
pelvis_out = Dense(1, activation='relu', name='pelvis_out')(dense)
leg_out = Dense(1, activation='relu', name='leg_out')(dense)
inner_leg_out = Dense(1, activation='relu', name='inner_leg_out')(dense)
thigh_out = Dense(1, activation='relu', name='thigh_out')(dense)
knee_out = Dense(1, activation='relu', name='knee_out')(dense)
calf_out = Dense(1, activation='relu', name='calf_out')(dense)


model = Model(
    inputs=inp,
    outputs=[
        head_out,
        neck_out,
        sts_out,
        arm_out,
        stw_out,
        torso_out,
        bicep_out,
        wrist_out,
        chest_out,
        waist_out,
        pelvis_out,
        leg_out,
        inner_leg_out,
        thigh_out,
        knee_out,
        calf_out
    ],
    name='conv_bodies'
)

model.summary()

In [ ]:
from keras.utils import plot_model

plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True, show_layer_activations=True)

In [0]:
#FOR BODY_M DATASET
# import csv
# 
# with open('../bodym-dataset/train/subject_to_photo_map.csv', 'r') as f:
#     reader = csv.reader(f)
#     uuid_raw = list(reader)
# from collections import defaultdict
# 
# uuid_map = defaultdict(list)
# 
# for subject_id, photo_id in uuid_raw[1:]:
#     uuid_map[subject_id].append(photo_id)
# image_data = {}
# 
# with open('../bodym-dataset/train/measurements.csv', 'r') as f:
#     reader = csv.reader(f)
#     
#     for line in reader:
#         subject_id = line[0]
#         
#         
#         for image_id in uuid_map[subject_id]:
#             image_data[image_id] = np.array(list(map(float, line[1:])))
#             
# import os
# 
# import cv2
# from tqdm import tqdm
# 
# raw_dataset = []
# 
# for image_id in tqdm(image_data):
#     img = cv2.imread(os.path.join('../bodym-dataset/train/mask', image_id + '.png'), cv2.IMREAD_GRAYSCALE)
#     img = cv2.resize(img, (180, 240))
#     
#     raw_dataset.append((
#         np.expand_dims(img, 2), 
#         image_data[image_id])
#     )

In [ ]:
import cv2
import os
from tqdm import tqdm

avatar_count = 79999
raw_dataset = {}

for number in tqdm(range(avatar_count)):
    if number == 5:
        continue
    source_image = cv2.imread(os.path.join(f"../Surreact-APose/train/imgs_nobg_front/Avatar_{number:06d}.png"), cv2.IMREAD_GRAYSCALE)
    silhouette = (source_image > 1).astype(np.bool_)
    raw_dataset[number] = (tuple(np.load(f"../Surreact-APose/train/bodymeasurements/Avatar_{number:06d}.npy", allow_pickle=True)), silhouette)

In [ ]:
X, y_true = map(np.array, zip(*raw_dataset))

In [ ]:
(
    y_true_head, y_true_neck, y_true_sts,y_true_arm, y_true_stw, y_true_torso, y_true_bicep, y_true_wrist, y_true_chest, y_true_waist, y_true_pelvis, y_true_leg, y_true_inner_leg, y_true_thigh, y_true_knee, y_true_calf
) = np.hsplit(y_true, 16)

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

model.compile(
    optimizer=Adam(),
    loss={
        'head_out': 'mse',
        'neck_out': 'mse',
        'shoulder_to_shoulder_out': 'mse',
        'arm_out': 'mse',
        'shoulder_to_wrist_out': 'mse',
        'torso_out': 'mse',
        'bicep_out': 'mse',
        'wrist_out': 'mse',
        'chest_out': 'mse',
        'waist_out': 'mse',
        'pelvis_out': 'mse',
        'leg_out': 'mse',
        'inner_leg_out': 'mse',
        'thigh_out': 'mse',
        'knee_out': 'mse',
        'calf_out': 'mse'
    },
    metrics={
        'head_out': ['mae'],
        'neck_out': ['mae'],
        'shoulder_to_shoulder _out': ['mae'],
        'arm_out': ['mae'],
        'shoulder_to_wrist_out': ['mae'],
        'torso_out': ['mae'],
        'bicep_out': ['mae'],
        'wrist_out': ['mae'],
        'chest_out': ['mae'],
        'waist_out': ['mae'],
        'pelvis_out': ['mae'],
        'leg_out': ['mae'],
        'inner_leg_out': ['mae'],
        'thigh_out': ['mae'],
        'knee_out': ['mae'],
        'calf_out': ['mae']
    },
)

model.fit(
    x=X,
    y={
        'head_out': y_true_head,
        'neck_out': y_true_neck,
        'shoulder_to_shoulder_out': y_true_sts,
        'arm_out': y_true_arm,
        'shoulder_to_wrist_out': y_true_stw,
        'torso_out': y_true_torso,
        'bicep_out': y_true_bicep,
        'wrist_out': y_true_wrist,
        'chest_out': y_true_chest,
        'waist_out': y_true_waist,
        'pelvis_out': y_true_pelvis,
        'leg_out': y_true_leg,
        'inner_leg_out': y_true_inner_leg,
        'thigh_out': y_true_thigh,
        'knee_out': y_true_knee,
        'calf_out': y_true_calf
    },
    batch_size=8,
    epochs=10,
    validation_split=0.2,
    shuffle=True,
    callbacks=[TensorBoard(write_graph=False)]
)

In [ ]:
model.save('models/abcd.keras')

In [ ]:
from keras.models import load_model

model = load_model('')